In [38]:
from PIL import Image
import numpy as np
import math
from time import time_ns

In [39]:
def upsampling (image: Image, k: int) -> Image:
  width, height = image.size

  pixels = image.load()

  newWidth = width * k
  newHeight = height * k
# Создаем пустое изображение с новыми размерами
  newPixels = np.random.random_integers(0, 255, (newHeight, newWidth, 3))
   # Преобразуем исходное изображение в массив для более быстрого доступа к пикселям
  newPixels = np.array(newPixels, dtype=np.uint8)
#Метод ближайшего соседа(вроде самый быстрый)
  for i in range(width):
    # для красоты выводим прогресс выполнения функции
    for j in range(height):
      for s_i in range(k):
        for s_j in range(k):
          # Присваиваем значение ближайшего пикселя пикселю нового изображения
          newPixels[j * k + s_j, i * k + s_i] = pixels[i, j]
      
  return Image.fromarray(newPixels)

In [40]:
def getImages():
    image_1 = Image.open('input/towelie.png', 'r').convert('RGB')
    image_2 = Image.open('input/Bilinear_interpolation.png', 'r').convert('RGB')
    image_3 = Image.open('input/kvant.png', 'r').convert('RGB')
    image_4 = Image.open('input/muar.png', 'r').convert('RGB')
    image_5 = Image.open('input/scls.png', 'r').convert('RGB')
    return {'first': image_1, 'second':image_2, 'third':image_3, 'fourth':image_4, 'fifth':image_5}

In [41]:
def printAudit(test_name, delta_time, audit):
  print(f'\nTest {test_name} passed: {(delta_time) / math.pow(10, 9)}s')
  for i in audit:
    name = i.get('name')
    input_size = i.get('input_size')
    output_size = i.get('output_size')
    print(f'Image {name}; input_size: {input_size}; output_size: {output_size}')

In [42]:
def upsampling_test():
  images = getImages()
  n = 5

  start_time = time_ns()
  audit = []
  for (image_name, image) in images.items():
    newImage = upsampling(image, n)
    output_name = f'result/{image_name}.processed.png'
    audit.append({
      'name': output_name,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{newImage.size[0]}x{newImage.size[1]}'
    })
    newImage.save(output_name)

  end_time = time_ns()
  printAudit('upsampling_test', end_time - start_time, audit)

In [43]:
# выбирается только один пиксель (верхний левый пиксель блока)
#для каждого блока n×n пикселей сначала извлекается весь блок пикселей, а затем вычисляется среднее значение по красному, зеленому и синему каналам отдельно для всех пикселей в блоке
def downsampling (image: Image, n: int, random_pick = False) -> Image:
  width, height = image.size

  pixels = image.load()

  newWidth = width // n
  newHeight = height // n

  newPixels = np.random.random_integers(0, 255, (newHeight, newWidth, 3))
  newPixels = np.array(newPixels, dtype=np.uint8)

  for i in range(newHeight):
    for j in range(newWidth):
      r, g, b = (0, 0, 0)

      x = j * n
      y = i * n

      # вычисление среднего значения (r, g ,b) пикселей в каждом блоке nxn
      if(random_pick):
        block = np.array(image.crop((x, y, x + n, y + n)))
        red, green, blue = list(), list(), list()

        for k in range(n):
          for m in range(n):
            red.append(block[k, m, 0])
            green.append(block[k, m, 1])
            blue.append(block[k, m, 2])

        r = int(np.mean(red))
        g = int(np.mean(green))
        b = int(np.mean(blue))
      else:
        # берем первый пиксель
        r, g, b = pixels[x, y]

      newPixels[i, j] = (r, g, b)
      
  return Image.fromarray(newPixels)

In [44]:
def downsampling_test():
  images = getImages()
  n = 5

  start_time = time_ns()
  audit = []
  for (image_name, image) in images.items():
    newImage = downsampling(image, n)
    output_name = f'output/downsampling/{image_name}.processed.png'
    audit.append({
      'name': output_name,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{newImage.size[0]}x{newImage.size[1]}'
    })
    newImage.save(output_name)

  end_time = time_ns()
  printAudit('downsampling_test', end_time - start_time, audit)

In [45]:
def resampling (image: Image, upsampling_k: int, downsampling_k: int) -> Image:
  return downsampling(upsampling(image, upsampling_k), downsampling_k)

In [46]:
def resampling_test():
  images = getImages()
  n = 3
  m = 7

  start_time = time_ns()
  audit = []
  for (image_name, image) in images.items():
    newImage = resampling(image, n, m)
    output_name = f'result/resampling/{image_name}.processed.png'
    audit.append({
      'name': output_name,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{newImage.size[0]}x{newImage.size[1]}'
    })
    newImage.save(output_name)

  end_time = time_ns()
  printAudit('resampling_test', end_time - start_time, audit)

In [47]:
def one_resampling (image: Image, m: float) -> Image:
  width, height = image.size
  pixels = image.load()
  #вычисляется соответствующая координата в исходном изображении путём деления координат пикселя нового изображения на коэффициент m и округления 
  newWidth = math.floor(width * m)
  newHeight = math.floor(height * m)

  newPixels = np.random.random_integers(0, 255, (newHeight, newWidth, 3))
  newPixels = np.array(newPixels, dtype=np.uint8)

  for i in range(newHeight):
    for j in range(newWidth):
      x = math.floor(j / m)
      y = math.floor(i / m)
      newPixels[i, j] = pixels[x, y]

  return Image.fromarray(newPixels)

In [48]:
def one_resampling_test():
  images = getImages()
  n = 64 / 235

  start_time = time_ns()
  audit = []
  for (image_name, image) in images.items():
    newImage = one_resampling(image, n)
    output_name = f'output/oneresampling/{image_name}.processed.png'
    audit.append({
      'name': output_name,
      'input_size': f'{image.size[0]}x{image.size[1]}',
      'output_size': f'{newImage.size[0]}x{newImage.size[1]}'
    })
    newImage.save(output_name)

  end_time = time_ns()
  printAudit('one_resampling_test', end_time - start_time, audit)


In [49]:
def main():
  upsampling_test()
  downsampling_test()
  resampling_test()
  one_resampling_test()

In [50]:
if __name__ == '__main__':
  main()

C:\Users\Kokovino\AppData\Local\Temp\ipykernel_14000\3513057684.py:9: DeprecationWarning: This function is deprecated. Please call randint(0, 255 + 1) instead
  newPixels = np.random.random_integers(0, 255, (newHeight, newWidth, 3))



Test upsampling_test passed: 12.0223295s
Image result/first.processed.png; input_size: 250x299; output_size: 1250x1495
Image result/second.processed.png; input_size: 320x306; output_size: 1600x1530
Image result/third.processed.png; input_size: 640x429; output_size: 3200x2145
Image result/fourth.processed.png; input_size: 317x245; output_size: 1585x1225
Image result/fifth.processed.png; input_size: 369x751; output_size: 1845x3755

Test downsampling_test passed: 0.0219563s
Image result/downsampling/first.processed.png; input_size: 250x299; output_size: 50x59
Image result/downsampling/second.processed.png; input_size: 320x306; output_size: 64x61
Image result/downsampling/third.processed.png; input_size: 640x429; output_size: 128x85
Image result/downsampling/fourth.processed.png; input_size: 317x245; output_size: 63x49
Image result/downsampling/fifth.processed.png; input_size: 369x751; output_size: 73x150


C:\Users\Kokovino\AppData\Local\Temp\ipykernel_14000\1630020670.py:11: DeprecationWarning: This function is deprecated. Please call randint(0, 255 + 1) instead
  newPixels = np.random.random_integers(0, 255, (newHeight, newWidth, 3))



Test resampling_test passed: 4.5779685s
Image result/resampling/first.processed.png; input_size: 250x299; output_size: 107x128
Image result/resampling/second.processed.png; input_size: 320x306; output_size: 137x131
Image result/resampling/third.processed.png; input_size: 640x429; output_size: 274x183
Image result/resampling/fourth.processed.png; input_size: 317x245; output_size: 135x105
Image result/resampling/fifth.processed.png; input_size: 369x751; output_size: 158x321

Test one_resampling_test passed: 0.0448845s
Image result/oneresampling/first.processed.png; input_size: 250x299; output_size: 68x81
Image result/oneresampling/second.processed.png; input_size: 320x306; output_size: 87x83
Image result/oneresampling/third.processed.png; input_size: 640x429; output_size: 174x116
Image result/oneresampling/fourth.processed.png; input_size: 317x245; output_size: 86x66
Image result/oneresampling/fifth.processed.png; input_size: 369x751; output_size: 100x204


C:\Users\Kokovino\AppData\Local\Temp\ipykernel_14000\1837066733.py:8: DeprecationWarning: This function is deprecated. Please call randint(0, 255 + 1) instead
  newPixels = np.random.random_integers(0, 255, (newHeight, newWidth, 3))
